In [6]:
import pandas as pd
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [3]:
datadir = '/opt/data'
localdir = '/opt/program'
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [5]:
dict(tripsdf.iloc[0])

{'tripduration': 634,
 'starttime': '2013-07-01 00:00:00',
 'stoptime': '2013-07-01 00:10:34',
 'start station id': 164,
 'start station name': 'E 47 St & 2 Ave',
 'start station latitude': 40.75323098,
 'start station longitude': -73.97032517,
 'end station id': 504,
 'end station name': '1 Ave & E 15 St',
 'end station latitude': 40.73221853,
 'end station longitude': -73.98165557,
 'bikeid': 16950,
 'usertype': 'Customer',
 'birth year': '\\N',
 'gender': 0}

In [7]:
datadir = f'/opt/program/artifacts/2020-07-08T143732Z'
proc_bundle = joblib.load(f'{datadir}/proc_bundle.joblib')

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.20.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
proc_bundle['proc_bundle'].keys()

dict_keys(['enc', 'usertype_le', 'le'])

In [11]:
record = {
 'starttime': '2013-07-01 00:00:00',
 'start station id': 164,
 'start station name': 'E 47 St & 2 Ave',
 'start station latitude': 40.75323098,
 'start station longitude': -73.97032517,
# unknown
# 'end station id': 504,
# 'end station name': '1 Ave & E 15 St',
# 'end station latitude': 40.73221853,
# 'end station longitude': -73.98165557,
# 'stoptime': '2013-07-01 00:10:34',
# 'tripduration': 634,
 'bikeid': 16950,
 'usertype': 'Customer',
 'birth year': '\\N',
 'gender': 0}

inputdf = pd.DataFrame.from_records([record])

##### Here was my pv1 recipe for preprocess + predict 


```python
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# preproc
(X_transformed,
     one_hot_enc, le,
     y_enc) = pv1.preprocess(X_train, y_train, # X[train_index]
                         neighborhoods)
labels = le.classes_

# Test set
X_test_transformed = one_hot_enc.transform(X_test)
y_test_enc = le.transform(y_test)


# predict
y_prob_vec = fu.predict_proba(X_test_transformed, bundle_loc=bundle_loc)

```

##### ok what if the bundle already exists 
* Actually pv1 doesnt handle existing preproc bundles ..

#### but pv2 does handle existing preproc bundles 
* the output is saved as a svm style file.. 
```python
import fresh.preproc.v2 as pv2

X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
X_train, X_test, y_train, y_test = train_test_split(X, y)


test_loc = pv2.preprocess(
        X_test, y_test, neighborhoods, proc_bundle=proc_bundle,
        workdir=workdir,
        dataset_name='test')
```

In [13]:
reload(fu)

<module 'fresh.utils' from '/opt/program/fresh/utils.py'>

In [21]:
!pwd

/opt/program


In [23]:
reload(fu)
rootdir = '/opt/program'
workdir = fu.make_work_dir(rootdir)
print(workdir)

/opt/program/artifacts/2020-08-19T144654Z


In [24]:
X = fu.prepare_data(inputdf, stationsdf, labelled=False)
neighborhoods = fu.neighborhoods_from_stations(stationsdf)

print(['start_neighborhood', 'gender', 'time_of_day', 'usertype', 'weekday', ])
print(X)

input_loc = pv2.preprocess(
        X=X, neighborhoods=neighborhoods, 
        proc_bundle=proc_bundle,
        workdir=workdir,
        dataset_name='input')
print(input_loc)

['start_neighborhood', 'gender', 'time_of_day', 'usertype', 'weekday']
[['Midtown East' 0 4 'Customer' 1]]


AssertionError: 

In [27]:
num_rows = X.shape[0]
print('num rows', num_rows)
# fu.get_slices(list(range(num_rows)), num_slices=10)
fu.get_slices([1], num_slices=1)


#fu.get_partitions(vec, slice_size, keep_remainder)

num rows 1


[[1, 2]]

In [ ]:
# Next... ^^  fix this pv2.preprocess AssertionError